In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns # heatmaps yay

from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from scipy.stats import skew
from scipy.stats import norm
from scipy.stats import boxcox

from scipy.special import inv_boxcox

import math

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
lmbda_opts = {}

In [ ]:
def drop_high_missing_features(df):
    tot_rec = len(df.index)
    for col in df.columns.values:
        if df[col].isnull().sum() / tot_rec > 0.15:
            del df[col]

def impute_missing_data(df):
    fill_with = {'None': ['PoolQC', 'MiscFeature', 'Alley', 'Fence',
                 'FireplaceQu', 'GarageType', 'GarageFinish',
                 'GarageQual', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
                 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType', 'MSSubClass'],
               0: ['GarageYrBlt', 'GarageArea', 'GarageCars',
                   'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                   'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                   'MasVnrArea'
                  ]}
    for fw in fill_with:
        for f in fill_with[fw]:
            df[f] = df[f].fillna(fw)
    
    
    df["Functional"] = df["Functional"].fillna("Typ")
    
    
    mode_list = ['SaleType', 'Exterior2nd', 'Exterior1st', 'KitchenQual', 
                 'Electrical', 'MSZoning']
    for f in mode_list:
        df[f] = df[f].fillna(df[f].mode()[0])
    
    
    df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    
    return df

def calculate_nulls(df):
    df_nulls = df.copy().isnull().sum()
    df_nulls = df_nulls.to_frame().rename(columns={0:'num_nulls'})
    df_nulls['total_records'] = len(df.index)
    df_nulls['pct_null'] = df_nulls['num_nulls'] / df_nulls['total_records'] * 100
    df_nulls = df_nulls[df_nulls['num_nulls'] > 0].sort_values(by='pct_null', ascending=False)
    return df_nulls

def process_df(df, is_test=False):
    df.set_index('Id')

    df = impute_missing_data(df)
    
    df = df.drop(['Utilities'], axis=1)

    # Convert to strings as they're categories:
    str_cols = ['MSSubClass', 'OverallCond', 'MoSold', 'YrSold', 'YearRemodAdd', 'YearBuilt', 'GarageYrBlt']
    for f in str_cols:
        df[f] = df[f].astype(str)

    ohe_cols = df.dtypes[df.dtypes == "object"].index
    df_dummies = pd.get_dummies(df[ohe_cols])
    df = df.drop(ohe_cols, axis='columns')
    df = pd.concat([df, df_dummies], sort=False, axis='columns')

    # Use a label encoder for the categorical fields
    le_cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
           'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
           'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
           'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
           'MoSold', 'YrSold', 'YearRemodAdd', 'YearBuilt', 'GarageYrBlt']
    for c in le_cols:
        if c in df.columns.values:
            le = LabelEncoder() 
            le.fit(list(df[c].values)) 
            df[c] = le.transform(list(df[c].values))
        
    numeric_feats = df.dtypes[df.dtypes != "object"].index.values        
    df = df.loc[:, numeric_feats]
        
    # Sale Price is in our training data, but not testing data
    shift = 1.0
    if 'SalePrice' in df.columns.values:
        filter = (df['GrLivArea'] > 4000) & (df['SalePrice'] < 200000)
        df = df.loc[~filter, :]

        # Check the skew of all numerical features
        skewed_feats = df[numeric_feats[1:]].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
        skewness = pd.DataFrame({'Skew' :skewed_feats})
        
        for f in skewness[abs(skewness) > 0.75].index:
            if f not in le_cols and '_' not in f: #ignore label encoded fields and OHE columns
                df[f], lmbda_opts[f] = boxcox(df[f] + shift)

    else:  # Test data
        for f in lmbda_opts:
            if f in df.columns:
                df[f] = boxcox(df[f] + shift, lmbda=lmbda_opts[f])
            elif f != 'SalePrice':
                df[f] = 0

    return df

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title = 'Confusion Matrix',
                          cmap = plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print('Normalized Confusion Matrix')
    else:
        print('Confusion Matrix Without Normalization')
    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i,j], fmt),
                horizontalalignment="center",
                color="white" if cm[i,j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_text_index(df, name):
    le = LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix

def show_results(clf, X, y):
    acc_score = clf.score(X, y)
    print('Accuracy: {}'.format(acc_score*100))
    
    y_pred = clf.predict(X)
    cm_matrix = confusion_matrix(y, y_pred)
    np.set_printoptions(precision=2)
    
    Class = encode_text_index(pd.DataFrame(y), y.columns[0])
    
    fig = plt.figure()
    plot_confusion_matrix(cm_matrix, classes=Class, normalize=True)
    plt.show()
    
    y_pred = clf.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(y, y_pred[:, 1])
    roc_auc = auc(fpr, tpr)
    
    fig = plt.figure()
    lw = 2
    
    plt.plot(fpr, tpr, color='red', lw=lw, label='ROC Curve (area = {:%0.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right', frameon=False)
    
    plt.show()
    

In [ ]:
def run_skm(model, X_train, y_train, X_test, y_test, X_pred, model_params={}):
    cn = type(model).__name__
    print(f"Model: {cn}")
    model.fit(X_train, y_train, **model_params)
    y_test_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_test_pred)
    model_score = model.score(X_train, y_train)
    print(f"Root Mean Squared Error: {math.sqrt(mse)} - Score: {model_score}")
    
    y_pred = model.predict(X_pred)
    
    df_sub = pd.DataFrame(y_pred, index=X_pred['Id'], columns=['SalePrice'])
    df_sub['SalePrice'] = inv_boxcox(df_sub['SalePrice'], lmbda_opts['SalePrice'])
    df_sub.to_csv(os.path.join('submissions', f"{cn}.{datetime.now():%Y%M%d_%H%m%s}.csv"))
    return y_pred
    

#### Import the train csv file to take a look at the data

In [ ]:
df = pd.read_csv(os.path.join('data','train.csv'))

In [ ]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
df.head()

How many records are we dealing with?

In [ ]:
len(df.index)

#### Determine which features are important

How much of each feature is null?

In [ ]:
calculate_nulls(df)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df.isnull(), cbar=False, ax=ax)

Another interesting note is that the GarageX type features all have the same amount of null values. This is likely because they're a part of the same records. We can verify this to see how many records have all those features set to null.

Next let's take a look at the correlations between each feature.

In [ ]:
plt.subplots(figsize = (30,20))
sns.heatmap(df.corr(), cmap="YlGnBu", annot=True);
plt.title("Heatmap of Feature Correlation", fontsize = 30);

Looking at the correlation heat map we can quickly identify features which are highly related to other features. For example 1stFlSF and TotalBsmntSF are highly correlated which makes sense because generally speaking most basements are full basements (the basement is wearing the first floor like a hat). The same goes for 1stFlSF and 2ndFlSF (This house is a mad hatter).

Another instance is GarageCars and GarageArea. This makes sense because you need more space to store more cars. This is the same rationale for TotRmsAbvGrd (Total rooms above ground) and GrLivArea (ground floor living area).

YearBuilt and GarageYrBlt also appear to be highly correlated. This makes sense because typically the house and the garage are built at the same time.

Let's take some time to graph some of these relationships and see what comes up.

In [ ]:
plt.subplots(figsize = (20,12))
sns.scatterplot(x=df['GrLivArea'], y= df['SalePrice']);

In [ ]:
filter = (df['GrLivArea'] > 4000) & (df['SalePrice'] < 200000)
df[filter][['GrLivArea', 'SalePrice']]

As we can see from the graph there are four possible outliers, two with a sale price over $700,000 and two with over 4500 sq. ft. but a much lower price. If we follow the trend of the graph the higher two outliers seem to fit the pattern and we can likely keep these two in however we should probably remove the other two outliers. As this is unique to the training dataset we'll do that below.

In [ ]:
df = df[~filter]

In [ ]:
mask = df['Electrical'].notna()
df = df.loc[(mask), :]

In [ ]:
sns.scatterplot(np.log(df['GrLivArea']), np.log(df['SalePrice']));

The Box-Cox test quickly and easily tells us if  we need to perform a transform on our data or not by telling us a lambda value:

* -1. is a reciprocal
* -.5 is a recriprocal square root
* 0.0 is a log transformation
* .5 is a square root transform and
* 1.0 is no transform.

It's important that our data be transformed into a normal distribution as most regression models require the data to be normally distributed.

In [ ]:
bc_list = ['LotArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'SalePrice']

for c in bc_list:
    xt, maxlog = boxcox(df[c] + 1)
    fig = plt.figure()
    sns.scatterplot(df[c], xt, alpha=0.5);

    print("{} lambda = {:g}".format(c, maxlog))
    
    #plt.show_legend()

Based on the Box-Cox test we ran on the columns from our dataset, we need to perform the following transformations in our `process_df` function:

- Log
  - LotArea
  - 1stFlrSF
  - 2ndFlrSF
  - SalePrice
  - GrLivArea

- Square root
  - TotalBsmtSF
  - GarageArea
  

Alternatively we can just use the transformed data (`xt` above) returned by the `boxcox` function.

In [ ]:
df = pd.read_csv(os.path.join('data','train.csv'))
df = process_df(df)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

y = df.pop('SalePrice')
X = df

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=8675309)

In [ ]:
X_train.dtypes

In [ ]:
features = ['LotArea', 
            '1stFlrSF',
            '2ndFlrSF',
            'GrLivArea',
            'TotalBsmtSF',
            'GarageArea',
            'YearBuilt',
            'FullBath',
            'BedroomAbvGr',
            'OverallQual']

# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
param = {'objective': 'reg:linear', 'max_depth': 100, 'lambda': 0.5, 'alpha': 0.5, 'eta': 1} #, 'eval_metric': 'auc'
evallist = [(dtrain, 'train'), (dtest, 'eval')]
num_round = 1000

In [ ]:
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)

In [ ]:
xgb.plot_importance(bst)

In [ ]:
xgb.plot_tree(bst)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
dtest = xgb.DMatrix(X_test)
ypred = bst.predict(dtest)

In [ ]:
mse = mean_squared_error(y_test, ypred)
print(f"Root Mean Squared Error: {math.sqrt(mse)}")

In [ ]:
bst.best_score

In [ ]:
df_pred = pd.read_csv(os.path.join('data','test.csv'))
df_pred = process_df(df_pred, is_test=True)

c_in_train = [c for c in df_pred.columns.values if c in X_train]
c_nin_pred = [c for c in X_train.columns.values if c not in df_pred]

df_pred = df_pred.loc[:, c_in_train]

for c in c_nin_pred:
    df_pred[c] = 0
    
df_pred = df_pred[X_train.columns]

In [ ]:
df_pred.head()

In [ ]:
dpred = xgb.DMatrix(df_pred)
ypred = bst.predict(dpred)

In [ ]:
df_sub = pd.DataFrame(ypred, index=df_pred['Id'], columns=['SalePrice'])

In [ ]:
df_sub['SalePrice'] = inv_boxcox(df_sub['SalePrice'], lmbda_opts['SalePrice'])

In [ ]:
now = datetime.now()
df_sub.to_csv(os.path.join('submissions', f'XGBoost_{now:%Y%m%d%H%M%S}.csv'))

## Scikit Learn Models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import PassiveAggressiveRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.tree import DecisionTreeRegressor


lr = LinearRegression()
en = ElasticNet()

rfr = RandomForestRegressor(n_estimators=100)
gbr = GradientBoostingRegressor()

dtr = DecisionTreeRegressor()



models = [lr, en, rfr, gbr, dtr]

In [ ]:
calculate_nulls(df_pred)

In [ ]:
df_pred.describe().loc['max'].T.sort_values(ascending=False)

In [ ]:
calculate_nulls(X_test)

In [ ]:
for model in models:
    try:
        run_skm(model, X_train, y_train, X_test, y_test, df_pred)
    except Exception as e:
        print(f'Error with model {model} - {e}')
        